In [22]:
import bioread
import pandas as pd
import heartpy as hp
import neurokit2 as nk

acq_file = bioread.read_file(r'C:\Users\User\Downloads\80_28.1.24 (NPU_template).acq')

channels_to_extract = ['ECG - ECG100C', 'C5 - Expression' ]
data = {}
for channel_name in channels_to_extract:
    channel = acq_file.named_channels[channel_name]
    data[channel_name] = channel.data

df = pd.DataFrame(data)
print(df)


         ECG - ECG100C  C5 - Expression
0             0.000000              0.0
1             0.000153              0.0
2            -0.000153              0.0
3             0.000000              0.0
4             0.000000              0.0
...                ...              ...
3132788      -0.097656              0.0
3132789      -0.100098              0.0
3132790      -0.102539              0.0
3132791      -0.105438              0.0
3132792      -0.109253              0.0

[3132793 rows x 2 columns]


In [23]:
df = df.dropna()
df = df.reset_index(drop=True)


In [24]:
non_zero_indices = df.index[df['C5 - Expression'] != 0]
index_rows_to_change = []
for i in range(len(non_zero_indices)-1):
    if non_zero_indices[i+1] == non_zero_indices[i]+1:
        index_rows_to_change.append(non_zero_indices[i])
for index in index_rows_to_change:
    df.loc[index, 'C5 - Expression'] = 0

In [25]:
print(df)

         ECG - ECG100C  C5 - Expression
0             0.000000              0.0
1             0.000153              0.0
2            -0.000153              0.0
3             0.000000              0.0
4             0.000000              0.0
...                ...              ...
3132788      -0.097656              0.0
3132789      -0.100098              0.0
3132790      -0.102539              0.0
3132791      -0.105438              0.0
3132792      -0.109253              0.0

[3132793 rows x 2 columns]


In [26]:
df_baseline = df[df['C5 - Expression'] == 99]

print(df_baseline)

        ECG - ECG100C  C5 - Expression
481122       0.016327             99.0


In [28]:
df_baseline_minute = pd.DataFrame()
for index in df_baseline.index:
    df_slice = df.iloc[index : index + 2000 * 60]
    df_baseline_minute = pd.concat([df_baseline_minute, df_slice])
print(df_baseline_minute)

        ECG - ECG100C  C5 - Expression
481122       0.016327             99.0
481123       0.016632              0.0
481124       0.017548              0.0
481125       0.017548              0.0
481126       0.018463              0.0
...               ...              ...
601117      -0.069733              0.0
601118      -0.070953              0.0
601119      -0.071259              0.0
601120      -0.072021              0.0
601121      -0.072632              0.0

[120000 rows x 2 columns]


In [2]:
ecg_data =  {
    "n_cue": [],
    "n_no_cue": [],
    "p_cue": [],
    "p_no_cue": [],
    "u_cue": [],
    "u_no_cue": []
}
for index, row in df.iterrows():
    value = row['C5 - Expression']
    ecg_value = row['ECG - ECG100C']
    
    
    if value == 020.0:
        ecg_data["n_cue"].append(ecg_value)
    if value == 030.0:
        ecg_data["n_no_cue"].append(ecg_value)
    if value == 120.0:
        ecg_data["p_cue"].append(ecg_value)
    if value == 130.0:
        ecg_data["p_no_cue"].append(ecg_value)
    if value == 220.0:
        ecg_data["u_cue"].append(ecg_value)
    if value == 230.0:
        ecg_data["u_no_cue"].append(ecg_value)
        
max_length = max(len(ecg_data[key]) for key in ecg_data)
for key in ecg_data:
    while len(ecg_data[key]) < max_length:
        ecg_data[key].append(None)
df_ecg = pd.DataFrame(ecg_data)
print(df_ecg)


    
         


         n_cue  n_no_cue     p_cue  p_no_cue     u_cue  u_no_cue
0    -0.092621 -0.018768 -0.009613 -0.011139  0.035706 -0.032196
1    -0.091858 -0.019073 -0.009308 -0.011597  0.036163 -0.031586
2    -0.091248 -0.018921 -0.009003 -0.011902  0.036774 -0.031433
3    -0.090790 -0.019073 -0.009460 -0.012665  0.037384 -0.031281
4    -0.089874 -0.018768 -0.009460 -0.012970  0.037689 -0.031128
...        ...       ...       ...       ...       ...       ...
1194       NaN  0.011444       NaN       NaN       NaN       NaN
1195       NaN  0.011902       NaN       NaN       NaN       NaN
1196       NaN  0.012360       NaN       NaN       NaN       NaN
1197       NaN  0.012817       NaN       NaN       NaN       NaN
1198       NaN  0.013428       NaN       NaN       NaN       NaN

[1199 rows x 6 columns]


In [4]:
## trying with HEARTPY df
ecg_sampling_rate = 2000
mean_heart_rate = {}
working_data, measures = hp.process(df["ECG - ECG100C"].values, ecg_sampling_rate)
mean_heart_rate["ECG - ECG100C"] = measures['bpm']



In [5]:
measures

{'bpm': 55.370887615384696,
 'ibi': 1083.6019176136363,
 'sdnn': 96.3206670362669,
 'sdsd': 57.10036487168562,
 'rmssd': 92.14341368092913,
 'pnn20': 0.823445318084346,
 'pnn50': 0.5911365260900643,
 'hr_mad': 59.0,
 'sd1': 65.15455829273219,
 'sd2': 119.220180224025,
 's': 24403.06920781976,
 'sd1/sd2': 0.5465061214494153,
 'breathingrate': 0.30157408560770455}

In [8]:
## trying with HEARTPY
import numpy as np 

ecg_sampling_rate = 2000
mean_heart_rate = {}
for column in df_ecg.columns:
    nmpy = np.array(df_ecg[column])
    working_data, measures = hp.process(nmpy, ecg_sampling_rate)
    mean_heart_rate[column] = measures['bpm']
    mean_heart_rate = {column: sum(heart_rates) / len(heart_rates) for column, heart_rates in mean_heart_rate.items()}
print("Mean Heart Rates:")
for column, mean_hr in mean_heart_rate.items():
    print(f"{column}: {mean_hr} bpm")

ValueError: negative dimensions are not allowed

In [6]:


## trying with neurokit   
ecg_sampling_rate = 2000
heart_rate_data = {}
for column in df_ecg.columns:
    signals = nk.ecg_process(df_ecg[column], ecg_sampling_rate)
    heart_rate = nk.ecg_rate(signals)
    heart_rate_data[column] = heart_rate
print("Heart Rates:")
for column, heart_rate in heart_rate_data.items():
    print(f"{column}: {heart_rate.mean()} bpm")

heart_rate_df = pd.DataFrame(heart_rate_data)

mean_heart_rate_per_column = heart_rate_df.mean(axis=0)

print("Mean Heart Rate per column:")
print(mean_heart_rate_per_column)
    
    







    

c:\users\user\appdata\local\programs\python\python38\lib\site-packages\neurokit2\ecg\ecg_clean.py:101: NeuroKitWarning: There are 299 missing data points in your signal. Filling missing values by using the forward filling method.
  warn(
c:\users\user\appdata\local\programs\python\python38\lib\site-packages\neurokit2\ecg\ecg_clean.py:153: FutureWarning: DataFrame.pad/Series.pad is deprecated. Use DataFrame.ffill/Series.ffill instead
  ecg_signal = pd.DataFrame.pad(pd.Series(ecg_signal))


TypeError: NeuroKit error: signal_smooth(): 'size' should be between 1 and length of the signal.

In [10]:
nmpy

array([-0.09262085, -0.09185791, -0.09124756, ...,         nan,
               nan,         nan])